# Dimensionality Reduction

- Now let's apply PCA to our dataset.
- We will develop our own implementation of PCA and then use scikit learn method. 
- We will also see the effect of (un) normalized data in PCA.

In [ ]:
# libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

np.set_printoptions(precision = 2, suppress=True)

## Generate example data

We are going to create random multivariate normal data specifying the correlation matrix between two dimensions.

In [ ]:
original_mean = [100, 1]
original_covariance = [[20, 3], [3, 2]]
X = np.random.multivariate_normal(original_mean, original_covariance, size=1000)

plt.scatter(X[:, 0], X[:, 1])
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

## Manual PCA

Recall PCA steps:

1. Compute covariance matrix
1. Compute eigenvectors and eigenvalues
1. Select top k eigenvalues (and eigenvectors)
1. Rotate original data with eigenvector matrix

In [ ]:
# 1. compute covariance matrix
# by default numpy.cov consider rows as variables, set rowvar=0
Sigma = np.cov(X, rowvar = 0)

# equivalent since covariance matrix is symmetric 
# Sigma2 = np.cov(np.transpose(X))

In [ ]:
# compare to original covariance matrix
print(X.shape)
print(Sigma.shape)
Sigma

In [ ]:
# 2. Compute eigenvalues and eigenvectors
eig_val, eig_vec = np.linalg.eig(Sigma)

In [ ]:
print(eig_val.shape)
print(eig_vec.shape)
eig_val

**(aside) Rotate original data with eigenvector matrix to see the rotation effect**

We may expect to see that data is now *aligned* with horizontal and vertical axis. 

In [ ]:
X_rotated = np.dot(X, eig_vec)

plt.scatter(X_rotated[:, 0], X_rotated[:, 1])
plt.xlabel("Feature 0")
plt.ylabel("Feature 1")
plt.show()

Question: how should the covariance matrix of rotated data be?

In [ ]:
np.cov(X_rotated, rowvar = 0)

Let's continue with PCA!

In [ ]:
# 2.b. according to the documentation eigenvalues does not need to be sorted, so let's do it
idx_sort = np.argsort(-eig_val)
print(idx_sort)

eig_val_sort = eig_val[idx_sort]
eig_vec_sort = eig_vec[:, idx_sort]

In [ ]:
# 3- Select top k eigenvalues (and eigenvectors)
# Let's plot eigenvalues and percentage of total variance explained

eig_val_sort_percentage = eig_val_sort / np.sum(eig_val_sort)
plt.bar(range(1,(len(eig_val)+1)), eig_val_sort_percentage); #!!!

In [ ]:
# in this case, with only two dimensions, 
# it is not very useful to plot the cumulative variance explained

eig_val_sort_cum = np.cumsum(eig_val_sort) / np.sum(eig_val_sort)
plt.bar(range(1,(len(eig_val)+1)), eig_val_sort_cum);

In this case, we only have two dimensions, so we would select only one dimension. As data is generated just from one distribution, the output is not very meaningfull. I leave here the code, and we will test it later. 

In [ ]:
TOP_EIGEN = 1
eig_val_short = eig_val_sort[0:TOP_EIGEN]
eig_vec_short = eig_vec_sort[:,0:TOP_EIGEN]

print(eig_val_short.shape)
print(eig_vec_short.shape)

In [ ]:
# 4. Rotate original data with reduced eigenvector matrix
X_pca = X.dot(eig_vec_short)
print(X_pca.shape)

In [ ]:
# we add some glitter to see the points
plt.scatter(X_pca, np.random.normal(scale=0.001, size=len(X_pca)))
plt.xlabel("Feature 0")
plt.show()

In [ ]:
print(np.mean(X, axis=0))
print(np.std(X, axis=0))

We see that data is not scaled, so it could affect on the results, let's scale it.

In [ ]:
# Normalize data
from sklearn import preprocessing
X_norm = preprocessing.scale(X, axis=0)

# check
print(np.mean(X_norm, axis=0))
print(np.std(X_norm, axis=0))

In [ ]:
# PCA

# 1. compute covariance matrix
Sigma = np.cov(X_norm, rowvar = 0)

# 2. Compute eigenvalues and eigenvectors
eig_val, eig_vec = np.linalg.eig(Sigma)

print (eig_val)
print (eig_vec)

# 2b. Sort eigenvalues
idx_sort = np.argsort(-eig_val)
eig_val_sort = eig_val[idx_sort]
eig_vec_sort = eig_vec[:, idx_sort]

# 3- Select top k eigenvalues (and eigenvectors)
eig_val_sort_percentage = eig_val_sort / np.sum(eig_val_sort)
plt.bar(range(1,(len(eig_val)+1)), eig_val_sort_percentage); 

TOP_EIGEN = 1
eig_val_short = eig_val_sort[0:TOP_EIGEN]
eig_vec_short = eig_vec_sort[:,0:TOP_EIGEN]

print(eig_val_short.shape)
print(eig_vec_short.shape)

In [ ]:
# plot rotated data (no dimension reduction)
X_norm_rotated = np.dot(X_norm, eig_vec_sort)
plt.scatter(X_norm_rotated[:, 0], X_norm_rotated[:, 1])
plt.xlabel("Feature 0")
plt.ylabel("Feature 1")
plt.show()

In [ ]:
# plot rotated data (dimension reduction) and glitter
X_norm_rotated = np.dot(X_norm, eig_vec_short)
plt.scatter(X_norm_rotated, np.random.normal(scale=0.001, size=len(X_norm_rotated)))
plt.xlabel("Feature 0")
plt.show()

# Real dataset

Now we will use a real dataset about wine to see PCA in action. 

In [ ]:
data = pd.read_csv('wine.csv', sep='\t')

In [ ]:
data.head()

We don't need the class for dimensionality reduction, so we construct a new dataframe without the column

In [ ]:
data2 = data.loc[:,~data.columns.isin(['class'])]
print(data.shape)
print(data2.shape)

### Exercise: Apply PCA to this new dataset.

- Compute the eigendecomposition and rotated data with reduced dimension. 
- Plot two first components and add the original class as color to see if the decomposition is having any effect. 

Answer the following questions:
- what is the dimension of the covariance matrix?
- what are the eigenvalues?
- how much variance is explained by the first two eigenvalues?

Note: save dimension-reduced data in variable `data_pca`

In [ ]:
# 1. compute covariance matrix
# 2. Compute eigenvalues and eigenvectors
# 3. Select top k eigenvalues (and eigenvectors)
# 4. Rotate original data with eigenvector matrix
# 5. Plot two first dimensions of rotated data with the color being the original label

In [ ]:
# to-do
# ...

In [ ]:
# ...

In [ ]:
# ...

## scikit-learn PCA

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
#pca_model = PCA()
#data_pca_sklearn = pca_model.fit_transform(data_norm)

In [ ]:
#print('Default number of components: {}'.format(pca_model.n_components_))
#print(pca_model.explained_variance_)
#print(pca_model.explained_variance_ratio_)
#print(np.cumsum(pca_model.explained_variance_ratio_))

In [ ]:
#pca_model = PCA(n_components = 5)
#data_pca_sklearn = pca_model.fit_transform(data_norm)
#
#print(pca_model.explained_variance_)
#print(pca_model.explained_variance_ratio_)
#print(np.cumsum(pca_model.explained_variance_ratio_))

In [ ]:
#print(data_pca_sklearn[0:2])
#print(data_pca[0:2])

In [ ]:
#plt.scatter(data_pca_sklearn[:,0], data_pca_sklearn[:,1], c=data['class'], cmap="plasma", linewidths=0);

#### [Home] Exercise
Apply your favourite(s) ML algorithm to estimate the 'class' with the other parameters. Test if you appreciate any difference when applying PCA to the features and later apply your model. 

- performance?
- computational time?
- memory requirements?

# END.